In [20]:
!../maslaconVenv/bin/python3.9 -m pip install pandas 
!../maslaconVenv/bin/python3.9 -m pip install numpy
!../maslaconVenv/bin/python3.9 -m pip install torch
!../maslaconVenv/bin/python3.9 -m pip install sentence_transformers
#!../maslaconVenv/bin/python3.9 -m pip install fasttext
!../maslaconVenv/bin/python3.9 -m pip install sklearn
!../maslaconVenv/bin/python3.9 -m pip install wordcloud
!../maslaconVenv/bin/python3.9 -m pip install matplotlib
#!../maslaconVenv/bin/python3.9 -m pip install pymorphy2
#!../maslaconVenv/bin/python3.9 -m pip install spacy-universal-sentence-encoder
!../maslaconVenv/bin/python3.9 -m pip install datasets
!../maslaconVenv/bin/python3.9 -m pip install -U accelerate
!../maslaconVenv/bin/python3.9 -m pip install -U transformers

zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9
zsh:1: no such file or directory: ../maslaconVenv/bin/python3.9


In [17]:
!../maslaconVenv/bin/python3.9 -m pip install --upgrade pip
!../maslaconVenv/bin/python3.9 -m pip install transformers

In [22]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from datasets import Dataset 
#import fasttext
#import fasttext.util
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
import pymorphy2
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [23]:
# Загрузка данных
def load_data(file_path):
    df = pd.read_csv(file_path, header=None)
    question = df.iloc[0, 0]  # Первый вопрос в первой строке
    answers = df.iloc[1:, 0].tolist()  # Ответы
    return question, answers

# Стырил из старой версии
def preprocess_text(text):
    morph = pymorphy2.MorphAnalyzer()
    # Приведение к нижнему регистру
    text = text.lower()
    # Сохранение отрицаний и усилений
    # Объединим "не" с последующим словом
    text = re.sub(r'\b(не|ни|нет)\s+', r'\1', text)
    text = re.sub(r'\b(очень|сильно|крайне)\s+', r'\1', text)
    # Удаление пунктуации и спецсимволов
    text = re.sub(r'[^\w\s]', '', text)
    # Токенизация
    words = text.split()
    # Лемматизация без удаления стоп-слов
    words = [morph.parse(word)[0].normal_form for word in words]
    # Сбор обратно в строку
    return ' '.join(words)

def to_lower(text):
    text = [word.lower() for word in text]
    return text

# Слепил из первой версии
def vectorize_tfidf(answers):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
    embeddings = vectorizer.fit_transform(answers)
    return embeddings

# Векторизация с помощью Sentence-BERT
def vectorize_sentence_bert(answers):
    model = SentenceTransformer('all-mpnet-base-v2')
    embeddings = model.encode(answers)
    return embeddings

# Векторизация с помощью дообученного Sentence-BERT
def vectorize_sentence_bert_ft(answers):
    model = SentenceTransformer('../models/sentence_bert_ft')
    embeddings = model.encode(answers)
    return embeddings

def sbert_fine_tuning():
    # Загружаем базовую модель
    model = SentenceTransformer('all-mpnet-base-v2')
    # Загрузка и подготовка данных
    df = pd.read_csv('../data/ftDataset.csv')
    # Создаем примеры для обучения
    train_examples = [InputExample(texts=[row['Ответ 1'], row['Ответ 2']], label=float(row['Схожесть'])) for index, row in df.iterrows()]
    # Создаем DataLoader для обучения
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    # Задаем loss-функцию (контрастивное обучение)
    train_loss = losses.CosineSimilarityLoss(model=model)
    # Дообучение модели
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)
    # Сохранение дообученной модели
    model.save('../models/sentence_bert_ft')

# Векторизация с помощью FastText
'''def vectorize_fasttext(answers):
    fasttext.util.download_model('ru', if_exists='ignore')  # Загрузка модели FastText для русского языка
    model = fasttext.load_model('cc.ru.300.bin')
    embeddings = [model.get_sentence_vector(answer) for answer in answers]
    return np.array(embeddings)'''

# Поиск оптимального количества кластеров с использованием метода "локтя"
def find_optimal_clusters_elbow(embeddings, max_clusters=10):
    distortions = []
    for i in range(2, max_clusters+1):
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(embeddings)
        distortions.append(kmeans.inertia_)
    
    # Определение оптимального количества кластеров (точка локтя)
    elbow_point = np.argmin(np.diff(distortions)) + 2
    return elbow_point

def find_optimal_clusters_silhouette(embeddings, max_clusters=10):
    silhouette_scores = []
    for i in range(2, max_clusters+1):
        kmeans = KMeans(n_clusters=i, random_state=42)
        labels = kmeans.fit_predict(embeddings)
        score = silhouette_score(embeddings, labels)
        silhouette_scores.append(score)
    
    # Определение оптимального количества кластеров (максимальный силуэтный коэффициент)
    optimal_clusters = np.argmax(silhouette_scores) + 2
    return optimal_clusters

# Вычисление попарных сходств с использованием Sentence-BERT
def calculate_similarity_sentence_bert(answers):
    model = SentenceTransformer('all-mpnet-base-v2')
    #model = SentenceTransformer('paraphrase-MiniLM-L12-v2')
    embeddings = model.encode(answers)
    
    # Вычисление попарного сходства с использованием косинусной меры
    similarity_matrix = cosine_similarity(embeddings)
    return embeddings, similarity_matrix

# Кластеризация на основе матрицы сходства с использованием иерархической кластеризации
def cluster_similarity(similarity_matrix, num_clusters):
    clustering = AgglomerativeClustering(n_clusters=num_clusters, linkage='complete')
    clusters = clustering.fit_predict(1 - similarity_matrix)  # Используем 1 - сходство для создания матрицы расстояний
    return clusters

# Кластеризация с использованием KMeans
def cluster_answers(embeddings, num_clusters=3):
    kmeans = KMeans(n_clusters=num_clusters, random_state=10)
    kmeans.fit(embeddings)
    clusters = kmeans.labels_
    return clusters


In [24]:
''' СОЗДАНИЕ ОБЛАКОВ '''

# Создание облаков слов для каждого кластера
def create_word_clouds(answers, clusters):
    unique_clusters = np.unique(clusters)
    summary = ''
    
    for cluster in unique_clusters:
        cluster_answers = [answers[i] for i in range(len(answers)) if clusters[i] == cluster]
        text = ' '.join(cluster_answers)
        wordcloud = WordCloud(width=800, height=400, background_color='#c5e8e7', contour_color='#c5e8e7',
                          font_path='./Domino/Domino Bold.otf').generate(text)
        clusterTitle = list(wordcloud.words_.keys())[0]
        summary += clusterTitle + ' - ' + str(len(cluster_answers)) + '\n'
        
        # Визуализация облака слов
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Облако \"{clusterTitle}\"')
        plt.show()

    print(summary)

# Создание облака слов для каждого кластера
def create_word_clouds_freq(answers, clusters):
    unique_clusters = set(clusters)
    summary = ''

    for cluster in unique_clusters:
        cluster_answers = [answers[i] for i in range(len(answers)) if clusters[i] == cluster]
        
        # Подсчитываем частоту каждого слова
        word_freq = Counter(cluster_answers)
        #print(word_freq.most_common()[0][0])
        
        # Создание облака слов
        wordcloud = WordCloud(width=800, height=400, background_color='#c5e8e7', contour_color='#c5e8e7',
                          font_path='./Domino/Domino Bold.otf').generate_from_frequencies(word_freq)
        
        clusterTitle = list(wordcloud.words_.keys())[0]
        summary += clusterTitle + ' - ' + str(len(cluster_answers)) + '\n'

        # Визуализация облака слов
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Облако \"{clusterTitle}\"')
        plt.show()

    print(summary)
    
# Создание общего облака слов
def create_general_word_cloud(answers):
    text = ' '.join(answers)
    wordcloud = WordCloud(width=800, height=400, background_color='#c5e8e7', contour_color='#c5e8e7',
                          font_path='./Domino/Domino Bold.otf').generate(text)
    
    # Визуализация облака слов
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Общее облако слов')
    plt.show()

# Создание общего облака слов
def create_general_word_cloud_freq(answers):
    word_freq = Counter(answers)
    wordcloud = WordCloud(width=800, height=400, background_color='#c5e8e7', contour_color='#c5e8e7',
                          font_path='./Domino/Domino Bold.otf').generate_from_frequencies(word_freq)
    
    # Визуализация облака слов
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Общее облако слов')
    plt.show()

In [25]:
''' СЛОВАРЬ и ОБЩЕЕ ОБЛАКО '''

# Определение самого популярного слова в кластере
def get_dicts(answers, clusters):
    unique_clusters = set(clusters)
    wordsCnt = {}
    clustersCnt = {}

    for cluster in unique_clusters:
        cluster_answers = [answers[i] for i in range(len(answers)) if clusters[i] == cluster]
        
        # Подсчитываем частоту каждого слова
        cnt = Counter(cluster_answers)
        #print(cnt.most_common()[0][0])
        wordsCnt[cnt.most_common()[0][0]] = cnt.most_common()[0][1]
        clustersCnt[cnt.most_common()[0][0]] = len(cluster_answers)

    return (wordsCnt, clustersCnt)

# Создание общего облака слов
def create_cluter_word_cloud_freq(wCnt, cCnt):

    for k, v in wCnt.items():
        print(k, '-', v)
    wordcloud = WordCloud(width=800, height=400, background_color='#c5e8e7', contour_color='#c5e8e7',
                          font_path='./Domino/Domino Bold.otf').generate_from_frequencies(wCnt)
    
    # Визуализация облака слов
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Общее облако самых популярных СЛОВ')
    plt.show()

    for k, v in cCnt.items():
        print(k, '-', v)
    wordcloud = WordCloud(width=800, height=400, background_color='#c5e8e7', contour_color='#c5e8e7',
                          font_path='./Domino/Domino Bold.otf').generate_from_frequencies(cCnt)
   
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Общее облако самых популярных КЛАСТЕРОВ')
    plt.show()

In [7]:
''' MAIN '''


question, answers = load_data('../data/reps3.csv')
print(f"Вопрос: {question}")

# TF-IDF
#answers = [preprocess_text(ans) for ans in answers]
#embTfidf = vectorize_tfidf(answers)

# BERT, Fasttext
embBert = vectorize_sentence_bert(answers)
#embFast = vectorize_fasttext(answers)

# Similarity BERT
#embBert, simMatr = calculate_similarity_sentence_bert(answers)

#nClusters = find_optimal_clusters_silhouette(embBert)
#nClusters = 6
nClusters = max(7, find_optimal_clusters_silhouette(embBert, 15))

# For BERT and Fasttext:
clusters = cluster_answers(embBert, nClusters)

# For Similarity BERT:
#clusters = cluster_similarity(simMatr, nClusters)

# Independent word clouds:
create_word_clouds_freq(answers, clusters)    
create_general_word_cloud_freq(answers)

create_cluter_word_cloud_freq(*get_dicts(answers, clusters))

# Дообучение и использование новой модели

FileNotFoundError: [Errno 2] No such file or directory: './reps3.csv'

In [26]:
# Отдельно дообучение и проверка на том же датасете
question, answers = load_data('../data/reps3.csv')
print(f"Вопрос: {question}")

sbert_fine_tuning()
embBert = vectorize_sentence_bert_ft(answers)
nClusters = max(7, find_optimal_clusters_silhouette(embBert, 15))
clusters = cluster_answers(embBert, nClusters)

create_word_clouds_freq(answers, clusters)    
create_general_word_cloud_freq(answers)

create_cluter_word_cloud_freq(*get_dicts(answers, clusters))

Вопрос: Что бы вы хотели видеть на фуршете в следующий раз?


/Users/nigger/Documents/maslacon/maslaconVenv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`